# Testing the Convex Networking Model

This notebook demonstrates how to load the `InConvexHull` custom layer, generate synthetic test data, build and train a simple model, and make predictions.

In [ ]:
!pip install -r requirements

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Import the custom layer from the convexNetworking.py file
try:
    from convexNetworking import InConvexHull
except ImportError:
    # If the file is not directly in the path, you might need to adjust the import
    # For example, if it's in a subdirectory named 'convexNetworking'
    import sys
    import os
    # Assuming convexNetworking.py is in the parent directory of this notebook
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
    from convexNetworking import InConvexHull

print("TensorFlow version:", tf.__version__)
print("Custom layer 'InConvexHull' imported successfully.")

TensorFlow version: 2.12.0
Custom layer 'InConvexHull' imported successfully.


## Generate Test Data

We will generate simple 3D data points that are either inside or outside a small region around the origin. Points inside the region will be labeled as 1, and points outside as 0.

In [5]:
np.random.seed(42)
num_samples = 1000
input_dim = 3

# Points inside a small cube around origin (label 1)
xs_inside = np.random.rand(num_samples // 2, input_dim) * 0.2 - 0.1 # Centered around 0, small range
ys_inside = np.ones((num_samples // 2, 1), dtype=np.float32)

# Points outside (label 0)
xs_outside = np.random.rand(num_samples // 2, input_dim) * 2.0 - 1.0 # Larger range
# Ensure some outside points are not accidentally inside the small cube
outside_mask = np.any(np.abs(xs_outside) > 0.1, axis=1)
xs_outside = xs_outside[outside_mask]
ys_outside = np.zeros((xs_outside.shape[0], 1), dtype=np.float32)

# Combine data
xs = np.concatenate([xs_inside, xs_outside], axis=0)
ys = np.concatenate([ys_inside, ys_outside], axis=0)

# Shuffle data
indices = np.arange(xs.shape[0])
np.random.shuffle(indices)
xs = xs[indices]
ys = ys[indices]

print(f"Generated {len(xs)} samples with {input_dim} dimensions.")
print(f"{np.sum(ys)} points labeled as inside (1).")

Generated 1000 samples with 3 dimensions.
500.0 points labeled as inside (1).


## Build and Compile the Model

We will create a simple Sequential model with the `InConvexHull` layer as the main component. The layer will learn the hyperplanes defining the convex hull.

In [6]:
# Define the model using the custom layer
# Let's try to learn a hull defined by a certain number of hyperplanes
num_hyperplanes_to_learn = 6 # For a 3D space, 6 planes can form a cube-like shape

model = keras.Sequential([
    # Input layer - specify input shape
    keras.layers.InputLayer(input_shape=(input_dim,)),
    # Add the custom convex hull layer
    InConvexHull(num_hyperplanes=num_hyperplanes_to_learn),
    # The custom layer outputs a score between 0 and 1, suitable for binary classification
])

# Compile the model
# Use BinaryCrossentropy as the loss function since the output is a score/probability for binary classification
model.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_convex_hull (InConvexHul  (None, 1)                24        
 l)                                                              
                                                                 
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________


## Train the Model

Now we train the model using the generated data. The model will learn the parameters of the hyperplanes to distinguish between points inside and outside the target region.

In [7]:
print("\nTraining the model...")
# You might need to adjust the number of epochs based on your data and desired accuracy
history = model.fit(xs, ys, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Training the model...
Epoch 1/50
25/25 [==============================] - 0s 6ms/step - loss: 1.3774 - accuracy: 0.5163 - val_loss: 0.9774 - val_accuracy: 0.4350
Epoch 2/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6190 - accuracy: 0.5163 - val_loss: 0.5112 - val_accuracy: 0.4350
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 0.3729 - accuracy: 0.8163 - val_loss: 0.3388 - val_accuracy: 0.9900
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 0.2650 - accuracy: 0.9912 - val_loss: 0.2553 - val_accuracy: 0.9900
Epoch 5/50
25/25 [==============================] - 0s 2ms/step - loss: 0.2073 - accuracy: 0.9862 - val_loss: 0.2043 - val_accuracy: 0.9900
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.1704 - accuracy: 0.9850 - val_loss: 0.1714 - val_accuracy: 0.9900
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: 0.1455 - accuracy: 0.9837 - val_loss: 0.1482 - val_accuracy: 0.98

## Evaluate the Model (Optional)

You can evaluate the trained model on the training or a separate test dataset.

In [8]:
# Evaluate the model on the training data
loss, accuracy = model.evaluate(xs, ys, verbose=0)
print(f"\nTraining Loss: {loss:.4f}, Training Accuracy: {accuracy:.4f}")


Training Loss: 0.0273, Training Accuracy: 0.9950


## Make Predictions

You can use the trained model to predict the 'hull score' for new data points. A score closer to 1 indicates that the point is likely inside the learned convex hull.

In [9]:
# Create some new test points
test_points = np.array([
    [0.0, 0.0, 0.0], # Inside
    [0.05, 0.05, 0.05], # Inside
    [0.5, 0.5, 0.5], # Outside
    [-0.2, 0.2, -0.2], # Outside
    [10.0, 10.0, 10.0] # Far outside
], dtype=np.float32)

# Predict the hull scores
predictions = model.predict(test_points)

print("\nHull scores for test points:")
for i, point in enumerate(test_points):
    print(f"Point: {point}, Hull Score: {predictions[i][0]:.4f}")

# You can set a threshold to classify points (e.g., score > 0.5)
threshold = 0.5
print(f"\nClassification based on threshold > {threshold}:")
for i, point in enumerate(test_points):
    is_inside = predictions[i][0] > threshold
    print(f"Point: {point}, Predicted Inside: {is_inside}")

1/1 [==============================] - 0s 55ms/step

Hull scores for test points:
Point: [0. 0. 0.], Hull Score: 0.9898
Point: [0.05 0.05 0.05], Hull Score: 0.9842
Point: [0.5 0.5 0.5], Hull Score: 0.0000
Point: [-0.2  0.2 -0.2], Hull Score: 0.3456
Point: [10. 10. 10.], Hull Score: 0.0000

Classification based on threshold > 0.5:
Point: [0. 0. 0.], Predicted Inside: True
Point: [0.05 0.05 0.05], Predicted Inside: True
Point: [0.5 0.5 0.5], Predicted Inside: False
Point: [-0.2  0.2 -0.2], Predicted Inside: False
Point: [10. 10. 10.], Predicted Inside: False
